In [1]:
# Class IT-533 Week #7  - Measures of Association
# Summer 2020
# Lab #6 - ramen-ratings.csv
# By Wei Geng(David) and Anusha Bale
# Created on: 06/28/2020
# Honor Code: “I have neither given or received, nor have I tolerated others' use of unauthorized aid.”

In [2]:
# install pandas in case you dont have pandas, matplotlib and sklearn pre-installed
%pip install pandas
%pip install matplotlib
%pip sklearn

In [3]:
# Convert to pandas DataFrame
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
remen_df = pd.read_csv('https://raw.githubusercontent.com/intelspeedstep/AMOD533/master/ramen-ratings.csv')
remen_df.head()

,Review #,Brand,Variety,Style,Country,Stars,Top Ten
0,2580,New Touch,T's Restaurant Tantanmen,Cup,Japan,3.75,NaN
1,2579,Just Way,Noodles Spicy Hot Sesame Spicy Hot Sesame Guan...,Pack,Taiwan,1,NaN
2,2578,Nissin,Cup Noodles Chicken Vegetable,Cup,USA,2.25,NaN
3,2577,Wei Lih,GGE Ramen Snack Tomato Flavor,Pack,Taiwan,2.75,NaN
4,2576,Ching's Secret,Singapore Curry,Pack,India,3.75,NaN


In [4]:
# output the data attributes and its datatypes, plus the shape of the dataset
print(remen_df.info(verbose=True))
print(remen_df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2580 entries, 0 to 2579
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Review #  2580 non-null   int64 
 1   Brand     2580 non-null   object
 2   Variety   2580 non-null   object
 3   Style     2578 non-null   object
 4   Country   2580 non-null   object
 5   Stars     2580 non-null   object
 6   Top Ten   41 non-null     object
dtypes: int64(1), object(6)
memory usage: 141.2+ KB
None
(2580, 7)


In [5]:
# Convert Stars from object to numberic and Review # to str since ID number usually has no analytical value
remen_df.loc[remen_df["Stars"]=='Unrated','Stars']=np.nan
remen_df['Stars']=remen_df['Stars'].astype(float)
remen_df['Review #']=remen_df['Review #'].astype(str)

### <font color = 'orange'> Model #1: Run my team dataset with Decision Tree and optimize the configuration for model comparisons</font>
</font></br>

In [6]:
# Convert the string variables to numeric categorical varibles, so the model can run. Sklearn classifers requires categorical variables to be numerically coded.
from sklearn.preprocessing import LabelEncoder
remen_df["Brand"] = LabelEncoder().fit_transform(remen_df["Brand"])
remen_df["Variety"] = LabelEncoder().fit_transform(remen_df["Variety"])
remen_df["Style"] = LabelEncoder().fit_transform(remen_df["Style"].astype(str))
remen_df["Country"] = LabelEncoder().fit_transform(remen_df["Country"])
remen_df["Top Ten"] = LabelEncoder().fit_transform(remen_df["Top Ten"].astype(str))
remen_df.head()

,Review #,Brand,Variety,Style,Country,Stars,Top Ten
0,2580,190,2189,4,18,3.75,38
1,2579,119,1443,5,32,1.00,38
2,2578,192,454,4,35,2.25,38
3,2577,336,709,5,32,2.75,38
4,2576,38,1954,5,16,3.75,38


In [7]:
# 1) Decision Tree can handle both continuous or binary variables, So, I decided to make 'Stars' as a binary target variable, which I need to convert it first from 
# continuous to binary. This is supervised learning for classification problem.
# 2) Note that Multicolinearity doesnt negatively impact any tree based algorithm, such as decision tree or random forest since these algorithms are not linear model. 
# so, we only need drop the ID variable. 
# 3) split dataset into training and testing with ratio of 70% to 30%.
# 4) Fit the model with all variables in X, and select depth of tree to 6. If depth is a small number, the model is likily to be  stable, but if dept is a big number, it could be more overfitting.
# I pick 6 this time. I think this is a reasonable number, not too big nor too small. Meanwhile, I contro the mininmum sample leaf to 50. Again, if you set the leaf too
# small, there is a risk of overfitting. I think 50 is a good number.
# 5) After running the model, we get model accuracy equal to 78% on training and 76% on the test dataset. This is a pretty good model.
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
remen_df.loc[remen_df['Stars']>3,'Stars_Ind']=1
remen_df['Stars_Ind'].fillna(0,inplace=True)
X=remen_df.drop(['Stars','Stars_Ind','Review #'], axis=1)
y=remen_df['Stars_Ind']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1859, test_size=0.3)
tree = DecisionTreeClassifier(max_depth=6,  min_samples_leaf=50, random_state=1859)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print("Test set predictions:\n", y_pred)
print("Training set score: {:.2f}".format(tree.score(X_train, y_train)))
print("Test set score: {:.2f}".format(tree.score(X_test, y_test)))

Test set predictions:
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 

### <font color = 'orange'> Model #2: Run my team dataset with Random Forest and optimize the configuration for model comparisons</font>
</font></br>

In [8]:
# 1) Random Forest is a probability tree algorthm which uses the ensemble method to build multiple number of decision trees to average out the performance
# This would give us a very stable model, however, it is not difficult to explain the which features are important in which model.
# 2) Again, Random Forest can handle both continuous and categorical target variable , 
# so we set 'Stars' as a binary target variable, and split data with hold-out methold with 70% to 30%. 
# 3) we also set n_estimator to 100, this means we would split dataset into 30 pieces and build 100 different trees. 
# 4) After running the model, we get model accuracy equal to 78% on the training set and 73% on test set. This is a very good model, but the model 
# generalization is not as good  as the decision tree. To solve this, we canto use some techiques in auto-tuning the model to find the best parameter values 
# that can optimize model, This is another machine learning topic. 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
X=remen_df.drop(['Stars','Stars_Ind','Review #'], axis=1)
y=remen_df['Stars_Ind']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1859, test_size=0.3)
forest = RandomForestClassifier(n_estimators=100, random_state=1859)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)
print("Test set predictions:\n", y_pred)
print("Training set score: {:.2f}".format(tree.score(X_train, y_train)))
print("Test set score: {:.2f}".format(forest.score(X_test, y_test)))

Test set predictions:
 [0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1.
 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0.
 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1.
 1. 0. 1. 1. 1. 1. 0. 1. 1. 

### <font color = 'orange'>model #3: Run my team dataset with K-Nearest Neighbors model, and optimize the configuration for model comparisons. This was done from the previous lab.</font>

In [9]:
# 1) KNN handles binary variables, so, I first convert 'Stars' to a binary value by set value greater than 3 equal to 1, else to 0.
# This 'Stars_Ind' become my target variable. Again, this is supervised learning for classification problem.
# 2) No need to check correlation ahead or drop any variables since KNN is not linear model
# 3) split dataset into training and testing with ratio of 70% to 30%.
# 4) Fit the model with all variables in X, and select n to 5. If n is a small number, the model is less stable, but if n is a big number, it would be more errors.
# so, I manually tune the model to pick 5 as the value for n. If you pick 4 or 6, the model accuracy would decline.
# 5) After running the model, we get model accuracy equal to 72% on test dataset. This is a pretty good model. 
# However, there is concern on overfitting since the model as much better performance on the training dataset -- 81%.
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
X=remen_df.drop(['Stars','Stars_Ind','Review #'], axis=1)
y=remen_df['Stars_Ind']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1859, test_size=0.3)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print("Test set predictions:\n", y_pred)
print("Training set score: {:.2f}".format(knn.score(X_train, y_train)))
print("Test set score: {:.2f}".format(knn.score(X_test, y_test)))

Test set predictions:
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1.
 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 1. 

### <font color = 'orange'>Model #4: Run my team dataset with Naive Bayes model, and optimize the configuration for model comparisons. This was done from the previous lab.  </font>



In [10]:
# 1) Naive Bayes handles binary and categorical dependent variable well if predictors are independent
# 2) Again, we set 'Stars_Ind' as a binary target variable, and split data with hold-out methold with 70% to 30%
# 3) There are 3 types of naive bayes models --  Binomial, multimial and Guassian, and we use them based on the following rules,
# In the case of predictors are categorical variables, such as counts or labels, a multinomial distribution can be used. 
# If the variables are binary, such as yes/no or true/false, a binomial distribution can be used. 
# If a variable is numerical, such as a measurement, often a Gaussian distribution is used. 
# 4) I choose to use Gaussian distribution since the predictors are numerical.
# 5) After running the model, we get model accuracy equal to 74% on test dataset (we are predicting yes or no, we switch the label of target variable by (1-0.24))
# Moreover, the performance on the training dataset is 76%. This indicates it is a very stable model.
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
remen_df.loc[remen_df['Stars']>3,'Stars_Ind']=1
remen_df['Stars_Ind'].fillna(0,inplace=True)
X=remen_df.drop(['Stars','Stars_Ind','Review #'], axis=1)
y=remen_df['Stars_Ind']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1859, test_size=0.3)
GNB = GaussianNB()
GNB.fit(X_train,y_train)
y_pred = GNB.predict(X_test)
print("Test set predictions:\n", y_pred)
print("Training set score: {:.2f}".format(1-GNB.score(X_train, y_train)))
print("Test set score: {:.2f}".format(1-GNB.score(X_test, y_test)))

Test set predictions:
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 

**Note: Please check the attached Word Document for the answers for this week's lab assignment questions. Thanks**

### <font color = 'orange'>------------------------------------------------------------------------END--------------------------------------------------------------------------------- </font>